In [12]:
import numpy as np
import jax.numpy as jnp
import numpy.random as nprandom
import matplotlib.pyplot as plt
from amp import MSE, norm_sq_corr, PAL, hausdorff
from amp.marginal_separable_jax import GAMP_full, SE_fixed_C_v1
import amp.changepoint_jax as changepoint_jax
import amp.signal_configuration
from tqdm.auto import tqdm
from amp.signal_configuration import C_to_chgpt, generate_C_distanced, unif_prior_to_η_ϕ
from amp.signal_priors import SignalPrior, SparseDiffSignal, SparseGaussianSignal, GaussianSignal
from amp.posterior import MAP, MAP_η, compute_posterior, posterior_over_η, η_to_ψ
from amp.marginal_separable_jax import GAMP_gaussian_clean, GAMP_real_data, q

p = 80
n = int(1.3 * p)
σ = 0.0001 # noise standard deviation
L = 3

possible_chgpt_configurations = [[0.3, 0.7]]
true_chgpt_locations = lambda n: (n * np.array(possible_chgpt_configurations[0])).astype(int)
C_true = η_to_ψ(np.array(true_chgpt_locations(n)), n)

signal_prior = GaussianSignal(np.eye(L))
B̃ = signal_prior.sample(p)
X = np.random.normal(0, np.sqrt(1/n), (n, p))
Θ = X @ B̃
assert Θ.shape == (n, L)
Y = changepoint_jax.q(Θ, C_true, σ).sample()

#  Run DCDP
X_unit = np.sqrt(n) * X
Y_unit = np.sqrt(n) * Y
print("Running dcdp...")
dcdp_est = amp.comparison.DCDP(X_unit, Y_unit, λ_list = [5], γ_list = [5])

Running dcdp...


In [13]:
dcdp_est

[]

In [14]:
n * 0.7

72.8

In [7]:
import numpy as np

### Import R interfaces
import rpy2
import rpy2.robjects as ro
## To aid in printing HTML in notebooks
import rpy2.ipython.html
# rpy2.ipython.html.init_printing()
from rpy2.robjects.packages import importr, data

from amp.performance_measures import hausdorff
utils = importr('utils')
base = importr('base')
# charcoal = importr('charcoal')
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

## Import DPDU (R package)
chgpts = importr('changepoints')

## Import DCDP (Python file in home directory)
import amp.DCDP_utils as DCDP_utils

λ_list = [5]
γ_list = [500]

n = Y_unit.shape[0]
# The algorithm requires dividing the training set into two parts, 
# so we do that here:
Y_train = Y_unit[np.arange(0, n, 2)]
Y_test = Y_unit[np.arange(1, n, 2)]
X_train = X_unit[np.arange(0, n, 2), :]
X_test = X_unit[np.arange(1, n, 2), :]
grid_n = 100
dcdp = DCDP_utils.dcdp_cv_grid_linear(grid_n, λ_list, γ_list, smooth = 2, 
                buffer = 2, step_refine = 1, buffer_refine = 2, lam_refine = 0.1)
cp_best, param_best, cp_best_cand = dcdp.fit((Y_train, X_train), (Y_test, X_test))

# Run the algorithm on the full dataset and return the best changepoint
best_lam, best_gamma = param_best
grid_n = min(dcdp.grid_n, n - 1)
step = n / (grid_n + 1)
grid = np.floor(np.arange(1, grid_n + 1) * step).astype(int)
cp_loc, obj = dcdp.dp_grid((Y_unit, X_unit), grid, best_lam, best_gamma)
cp_loc_refined = dcdp.local_refine((Y_unit, X_unit), cp_loc)

/Users/gabrielarpino/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.560e-01, tolerance: 6.850e-02
  model = cd_fast.enet_coordinate_descent(
/Users/gabrielarpino/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.946e-02, tolerance: 9.134e-02
  model = cd_fast.enet_coordinate_descent(
/Users/gabrielarpino/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

In [8]:
cp_best

[24, 42]

In [9]:
cp_loc_refined

[52, 75, 87, 96]

## Run DP

In [5]:
import numpy as np
import jax.numpy as jnp
import numpy.random as nprandom
import matplotlib.pyplot as plt
from amp import MSE, norm_sq_corr, PAL, hausdorff
from amp.marginal_separable_jax import GAMP_full, SE_fixed_C_v1
import amp.changepoint_jax as changepoint_jax
import amp.signal_configuration
from tqdm.auto import tqdm
from amp.signal_configuration import C_to_chgpt, generate_C_distanced, unif_prior_to_η_ϕ
from amp.signal_priors import SignalPrior, SparseDiffSignal, SparseGaussianSignal, GaussianSignal
from amp.posterior import MAP, MAP_η, compute_posterior, posterior_over_η, η_to_ψ
from amp.marginal_separable_jax import GAMP_gaussian_clean, GAMP_real_data, q

p = 50
n = int(0.75 * p)
σ = 0.0001 # noise standard deviation
L = 3
Δ = 2
λ_list = [5]
γ_list = [100]

possible_chgpt_configurations = [[0.3, 0.7]]
true_chgpt_locations = lambda n: (n * np.array(possible_chgpt_configurations[0])).astype(int)
C_true = η_to_ψ(np.array(true_chgpt_locations(n)), n)

signal_prior = GaussianSignal(np.eye(L))
B̃ = signal_prior.sample(p)
X = np.random.normal(0, np.sqrt(1/n), (n, p))
Θ = X @ B̃
assert Θ.shape == (n, L)
Y = changepoint_jax.q(Θ, C_true, σ).sample()

#  Run DCDP
X_unit = np.sqrt(n) * X
Y_unit = np.sqrt(n) * Y
dp_est = amp.comparison.DP(X_unit, Y_unit, Δ, λ_list, γ_list)

In [6]:
dp_est

array([13.])

## Run DPDU

In [1]:
import numpy as np
import jax.numpy as jnp
import numpy.random as nprandom
import matplotlib.pyplot as plt
from amp import MSE, norm_sq_corr, PAL, hausdorff
from amp.marginal_separable_jax import GAMP_full, SE_fixed_C_v1
import amp.changepoint_jax as changepoint_jax
import amp.signal_configuration
from tqdm.auto import tqdm
from amp.signal_configuration import C_to_chgpt, generate_C_distanced, unif_prior_to_η_ϕ
from amp.signal_priors import SignalPrior, SparseDiffSignal, SparseGaussianSignal, GaussianSignal
from amp.posterior import MAP, MAP_η, compute_posterior, posterior_over_η, η_to_ψ
from amp.marginal_separable_jax import GAMP_gaussian_clean, GAMP_real_data, q

p = 80
n = int(1.25 * p)
σ = 0.0001 # noise standard deviation
L = 3
Δ = 2
λ_list = [5]
γ_list = [100]

possible_chgpt_configurations = [[0.3, 0.7]]
true_chgpt_locations = lambda n: (n * np.array(possible_chgpt_configurations[0])).astype(int)
C_true = η_to_ψ(np.array(true_chgpt_locations(n)), n)

signal_prior = GaussianSignal(np.eye(L))
B̃ = signal_prior.sample(p)
X = np.random.normal(0, np.sqrt(1/n), (n, p))
Θ = X @ B̃
assert Θ.shape == (n, L)
Y = changepoint_jax.q(Θ, C_true, σ).sample()

#  Run DCDP
X_unit = np.sqrt(n) * X
Y_unit = np.sqrt(n) * Y
dpdu_est = amp.comparison.DPDU(X_unit, Y_unit, λs = [0.3, 0.5, 1, 2], ζs = [10.0, 15.0, 20.0])

In [7]:
dpdu_est[-1]

array([20., 45., 73.])